In [1]:
import torch
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
import model
import datasets

In [ ]:
# Make sure to use the GPU. The following line is just a check to see if GPU is availables
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# load your dataset and dataloader
bird_dataset = datasets.bird_dataset()

# Fill in optional arguments to the dataloader as you need it
bird_dataloader = DataLoader(bird_dataset)

In [ ]:
# Create NN model object
nn_model = model.my_Net()
nn_model.to(device)


In [ ]:
# Create loss functions, optimizers


In [ ]:
# train your model
# For each epoch iterate over your dataloader/dataset, pass it to your NN model, get output, calculate loss and
# backpropagate using optimizer


In [ ]:
# Save your model/best model